In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Dependencies


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
!unzip ../input/facial-keypoints-detection/training.zip

In [ ]:
df = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
train = pd.read_csv('./training.csv')
key_points = train.drop(['Image'],axis=1)

## 2. Extracting images

In [ ]:
def extract_image(train):

    image_list = []

    for i in range(train.shape[0]):
        img = train['Image'][i].split(' ')
        
        img = np.array(list(map(int,img)))

        image_list.append(img)
    image_array = np.array(image_list).reshape((-1,96,96,1))
    return image_array

In [ ]:
image_array = extract_image(train)

In [ ]:
def show_img_and_points(df,index):
    '''A function to display image along with facial key points'''
    temp_point = df.iloc[index]

    x = []
    y = []
    points = df.iloc[index]
    for i in range(len(points)):
        if i%2 == 0:
            x.append(points[i])
        else:
            y.append(points[i])

    plt.imshow(image_array[index].reshape((96,96)),cmap='gray')
    plt.scatter(x,y,color='red')
    plt.show()

In [ ]:
show_img_and_points(key_points,0)

#### we can see 7048th data point has missing data and the plot below is evident of the fact, that only 4 points are available.

In [ ]:
print(key_points['left_eye_inner_corner_x'].isnull())
show_img_and_points(key_points,7048)

In [ ]:
feature_names = key_points.columns
feature_x = [i for i in feature_names if 'x' in i]
feature_y = [i for i in feature_names if 'y' in i]

In [ ]:
temp_point = key_points.iloc[7048]

x = []
y = []
points = key_points.iloc[7048]
for i in range(len(points)):
    if i%2 == 0:
        x.append(points[i])
    else:
        y.append(points[i])
        
plt.imshow(image_array[7045].reshape((96,96)),cmap='gray')
plt.scatter(x,y,color='red')

In [ ]:
def fill_missing_data(key_points,feature):
    
    nose_mean_x = key_points['nose_tip_x'].mean()
    nose_mean_y = key_points['nose_tip_y'].mean()
    
    if feature in feature_x:
      
        feature_mean = key_points[feature].mean()
    
        row_index_with_missing_value = key_points[feature][key_points[feature].isnull()].index
        for i in row_index_with_missing_value:
            error_i = key_points['nose_tip_x'].iloc[i] - nose_mean_x 
            key_points[feature].iloc[i] = feature_mean + error_i
            if key_points[feature].iloc[i] > 96 or key_points[feature].iloc[i] < 0:
                key_points[feature].iloc[i] = feature_mean - 0.5
            
    else:
        
        feature_mean = key_points[feature].mean()
    
        row_index_with_missing_value = key_points[feature][key_points[feature].isnull()].index
        for i in row_index_with_missing_value:
            error_i = key_points['nose_tip_y'].iloc[i] - nose_mean_x 
            key_points[feature].iloc[i] = feature_mean + error_i
            if key_points[feature].iloc[i] > 96 or key_points[feature].iloc[i] < 0:
                key_points[feature].iloc[i] = feature_mean - 0.5
            
           
            
    
   
        
    

In [ ]:
key_points_new = key_points.copy()

In [ ]:
for i in feature_names:
    fill_missing_data(key_points_new,i)
    print(i)
    
    

In [ ]:
print('key_points_new',key_points_new.isnull().sum())

### No Missing Values present now, hence we can feed it to our model

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Activation, Dropout, MaxPool2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping
from keras.models import load_model

In [ ]:
#callback = EarlyStopping(monitor='val_loss',restore_best_weights=True)

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())


model.add(Conv2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3),padding='same', use_bias=False))

model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Conv2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
Y_train = key_points_new.values
X_train = image_array

In [ ]:
model.fit(X_train,Y_train,epochs = 50,batch_size = 256,validation_split=0.2)

In [ ]:
!unzip ../input/facial-keypoints-detection/test.zip

In [ ]:
model.save('weigts3.h5')


In [ ]:
model = load_model('weigts3.h5')

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_img_array = extract_image(test)

In [ ]:
test_img_array.shape

In [ ]:
predictions = model.predict(test_img_array)

In [ ]:
predictions[0]

In [ ]:
points = predictions[24]
x = []
y = []
for i,p in enumerate(points):
    if i%2 == 0:
        x.append(p)
    else:
        y.append(p)
        
        
plt.imshow(test_img_array[24].reshape(96,96),cmap='gray')
plt.scatter(x,y,color='red')
        

In [ ]:
#predictions_submit = predictions.reshape(-1,1)

In [ ]:
submission = pd.DataFrame(predictions,columns=feature_names)

In [ ]:
submission = pd.concat([test['ImageId'],submission],axis=1)

In [ ]:
submission

In [ ]:
lookid = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')



In [ ]:

for i in range(lookid.shape[0]):
    feature = lookid['FeatureName'].iloc[i]
    img_id = lookid['ImageId'].iloc[i]
    location = submission[feature][submission['ImageId']==img_id].values[0]
#     print(feature,img_id,location,sep='--')
    lookid['Location'].iloc[i] = location
   

In [ ]:
def submission_prepare(lookid):
    for i in range(lookid.shape[0]):
        feature = lookid['FeatureName'].iloc[i]
        img_id = lookid['ImageId'].iloc[i]
        location = submission[feature][submission['ImageId']==img_id].values[0]
#     print(feature,img_id,location,sep='--')
        lookid['Location'].iloc[i] = location
    
    return lookid



In [ ]:
lookid.iloc[i]['Location'] =0 

In [ ]:
new = submission_prepare(lookid)

In [ ]:
new.to_csv('submission_new2.csv',index=False)

In [ ]:
predictions[predictions>96] = 96

In [ ]:
predictions[predictions>96]

In [ ]:
(new['Location']>96).sum()

In [ ]:
from IPython.display import FileLink
FileLink(r'./weigts3.h5')

In [ ]:
!cd